In [1]:
import os
import numpy as np
import nltk
from keras.models import Sequential, Model
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding,Dropout, Bidirectional, Input, merge
from keras.preprocessing.sequence import pad_sequences
import keras
import json
from data_utils import *
from model import *

C:\Users\Gzhang\AppData\Local\Continuum\Anaconda2\envs\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 10000

def load_cornell_movie_data(start_line, limit, vocab_dict):
    sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, start_line, limit)
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

def load_twitter_data(start_line, limit, vocab_dict):
    sentences = read_sentences("ShortenTwitterAsciiCorpus.txt")[1000:2000]
    x_vec, y_vec = vectorize_sentence(sentences, vocab_dict, VOCAB_SIZE, SENTENCE_LENGTH)
    return x_vec, y_vec

In [3]:
#model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 100, 3)
model = create_model_with_attention(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 300, 100, 3)
print(model.summary())
#model.load_weights("0new_model.h5")

C:\Users\Gzhang\Project\MLPractice\chatbot\model.py:29: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  output_att = merge([inputs, att_prob ], mode='mul')
C:\Users\Gzhang\AppData\Local\Continuum\Anaconda2\envs\python36\lib\site-packages\keras\legacy\layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 300)      3000000     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 20, 200)      240600      embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 20, 200)      40200       bidirectional_1[0][0]            
__________________________________________________________________________________________________
merge_1 (M

In [4]:
vocab_dict, idx_dict = load_vocab("vocab_dict.json")

In [ ]:
batch_size = 1000
start_point = 000
batch = 0
round = 0
while True:
    print("Getting data:", start_point, batch_size)
    x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
    print(x_vec.shape)
    if len(x_vec) == 0:
        start_point = 0
        batch = 0
        x_vec, y_vec = load_cornell_movie_data(start_point, batch_size, vocab_dict)
        round += 1 
    model.fit(x_vec, y_vec, batch_size=100, nb_epoch=100, shuffle = True)
    model.save_weights(str(batch%2)+"new_model.h5")
    start_point += batch_size
    batch += 1
    print("--------------------------------------------------------------------------")
    print("Round:", round, "Batch:", batch)
    print("--------------------------------------------------------------------------")

Getting data: 0 1000
(434, 20, 10000)
(434, 20)
Epoch 1/100


C:\Users\Gzhang\AppData\Local\Continuum\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


434/434 [==============================] - 10s 23ms/step - loss: 4.5356 - acc: 0.4019
Epoch 2/100
434/434 [==============================] - 10s 22ms/step - loss: 4.2620 - acc: 0.4156
Epoch 3/100
434/434 [==============================] - 10s 22ms/step - loss: 4.0922 - acc: 0.4160
Epoch 4/100
434/434 [==============================] - 10s 23ms/step - loss: 3.9955 - acc: 0.4173
Epoch 5/100
434/434 [==============================] - 10s 22ms/step - loss: 3.8938 - acc: 0.4207
Epoch 6/100
434/434 [==============================] - 10s 22ms/step - loss: 3.7600 - acc: 0.4237
Epoch 7/100
434/434 [==============================] - 10s 22ms/step - loss: 3.6932 - acc: 0.4262
Epoch 8/100
434/434 [==============================] - 10s 22ms/step - loss: 3.6199 - acc: 0.4274
Epoch 9/100
434/434 [==============================] - 10s 22ms/step - loss: 3.5861 - acc: 0.4300
Epoch 10/100
434/434 [==============================] - 10s 22ms/step - loss: 3.5154 - acc: 0.4270
Epoch 11/100
434/434 [=========

In [ ]:
#do you clean
sen = "what do you like to do"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '').split(" ")
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)

vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0] if idx_dict[x]!= 'UNK'])